In [69]:
# Install the necessary libraries
# !pip install xgboost
# !pip install seaborn


from google.colab import drive
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import xgboost as xgb

In [70]:
# Download The Dataset
drive.mount('/content/drive')
file_id = '1Afrm6F7MTAGwW55HBZdnNaTjt2HXNkK3'
destination = '/content/data.csv'
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)
data=pd.read_csv(destination)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading...
From: https://drive.google.com/uc?id=1Afrm6F7MTAGwW55HBZdnNaTjt2HXNkK3
To: /content/data.csv
100%|██████████| 76.6M/76.6M [00:01<00:00, 48.4MB/s]


In [71]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [72]:
data.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,1.047300e+06,1.047300e+06,1.047300e+06,1.047300e+06,1.047300e+06,1.047300e+06,1.047300e+06,1047300.0
mean,2.688335e+01,1.587272e+05,8.748024e+05,8.946201e+05,9.784019e+05,1.114510e+06,1.084694e-03,0.0
std,1.545126e+01,2.649332e+05,2.973320e+06,3.009850e+06,2.296836e+06,2.416662e+06,3.291684e-02,0.0
min,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,1.500000e+01,1.216573e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,2.000000e+01,7.645370e+04,1.600100e+04,0.000000e+00,1.267380e+05,2.186381e+05,0.000000e+00,0.0
75%,3.900000e+01,2.138531e+05,1.368815e+05,1.748382e+05,9.164329e+05,1.150567e+06,0.000000e+00,0.0
max,9.500000e+01,1.000000e+07,3.890000e+07,3.890000e+07,4.210000e+07,4.220000e+07,1.000000e+00,0.0


In [73]:
data.isnull().sum()

,0
step,0
type,0
amount,0
nameOrig,0
oldbalanceOrg,0
newbalanceOrig,0
nameDest,0
oldbalanceDest,0
newbalanceDest,0
isFraud,0


In [74]:
data.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [75]:
data['balanceOrig_diff'] = data['oldbalanceOrg'] - data['newbalanceOrig']
data['balanceDest_diff'] = data['oldbalanceDest'] - data['newbalanceDest']
data = pd.get_dummies(data, columns=['type'], drop_first=True)
data.drop(['nameOrig', 'nameDest'], axis=1, inplace=True)

In [76]:
# Split the data into training and testing sets
X = data.drop(['isFraud', 'isFlaggedFraud'], axis=1)
y = data['isFraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [77]:
# Build and train the XGBoost model with default parameters
model = xgb.XGBClassifier(eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [78]:
# Make predictions
y_pred = model.predict(X_test)

In [79]:
# Evaluate the model
confusion_matrix(y_test, y_pred)
classification_report(y_test, y_pred)
roc_auc_score(y_test, y_pred)

0.9195874570190866

In [80]:
result = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
result

,Actual,Predicted
487307,0,0
214116,0,0
493101,0,0
349977,0,0
811437,0,0
...,...,...
16670,0,0
624749,0,0
860368,0,0
662440,0,0
